## Import Libraries

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import os
import matplotlib as mpl
from PIL import Image
from palettable.cmocean.sequential import Deep_20
deep = mcolors.ListedColormap(Deep_20.mpl_colors)
import glob
from matplotlib import colors
from matplotlib.colors import LogNorm
import metpy.calc as calc
from metpy.units import units
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib.colors import TwoSlopeNorm
# from matplotlib.colors import DivergingNorm
import h5py
from matplotlib.lines import Line2D

In [ ]:
style = "/home/cneumaie/Scripts/Styles/christine-paperlight.mpstyle"

plt.style.use(style)
from matplotlib import rcParams

###Figure Settings
# -- Fonts --
rcParams['font.size'] = 35

In [ ]:
alt = np.array([  -25,    25,    75,   125,   175,   225,   275,   325,   375,
         425,   475,   525,   575,   625,   675,   725,   775,   825,
         875,   925,   975,  1025,  1075,  1125,  1175,  1225,  1275,
        1325,  1375,  1425,  1475,  1525,  1575,  1625,  1675,  1725,
        1775,  1825,  1875,  1925,  1975,  2025,  2075,  2125,  2175,
        2225,  2275,  2325,  2375,  2425,  2475,  2525,  2575,  2625,
        2675,  2725,  2775,  2825,  2875,  2925,  2975,  3025,  3075,
        3125,  3175,  3225,  3275,  3325,  3375,  3425,  3475,  3525,
        3575,  3625,  3675,  3725,  3775,  3825,  3875,  3925,  3975,
        4025,  4077,  4129,  4183,  4237,  4293,  4349,  4407,  4467,
        4527,  4589,  4651,  4715,  4781,  4847,  4915,  4985,  5055,
        5127,  5201,  5276,  5352,  5430,  5510,  5591,  5674,  5758,
        5844,  5932,  6022,  6114,  6207,  6302,  6399,  6498,  6599,
        6702,  6807,  6914,  7023,  7134,  7248,  7364,  7483,  7604,
        7727,  7852,  7980,  8111,  8244,  8380,  8519,  8660,  8805,
        8952,  9103,  9256,  9413,  9572,  9735,  9900, 10069, 10242,
       10418, 10597, 10780, 10967, 11156, 11350, 11549, 11749, 11949,
       12149, 12349, 12549, 12749, 12949, 13149, 13349])

In [ ]:
alt = alt[:99]

## Get Data for Figures

In [ ]:
filePaths = sorted(glob.glob(f"/moonbow/cneumaie/CP_train_derecho/processed_tke_v2/*"))
temp_list_cp1 = [10]
temp_list_cp2 = [5,10,20]
temp_list = [5,10,20]
time_list = [30,60,90, 120]
wind_list = ['shear', 'u0']
uprime_dataset = np.zeros((2,3,3,4,50,99,2998))
uprime_dataset[:] = np.nan
vprime_dataset = np.zeros((2,3,3,4,50,99,2998))
vprime_dataset[:] = np.nan
wprime_dataset = np.zeros((2,3,3,4,50,99,2998))
wprime_dataset[:] = np.nan
tke_dataset = np.zeros((2,3,3,4,50,99,2998))
tke_cpedge = np.zeros((2,3,3,4,50,99,300,3))
tke_cpedge[:] = np.nan
uprime_cpedge = np.zeros((2,3,3,4,50,99,300, 3))
uprime_cpedge[:] = np.nan
vprime_cpedge = np.zeros((2,3,3,4,50,99,300, 3))
vprime_cpedge[:] = np.nan
wprime_cpedge = np.zeros((2,3,3,4,50,99,300, 3))
wprime_cpedge[:] = np.nan

for filePath in filePaths:
    outfilename = filePath
    if "30" in outfilename:
        cp2_start = 8
    elif "60" in outfilename:
        cp2_start = 14
    elif "90" in outfilename:
        cp2_start = 20
    elif "120" in outfilename:
        cp2_start = 26 

    ###Loop through all variations and perform depth and environment calculations for the case that matches the filePath
    for w,wind in enumerate(wind_list):
        for c,CP1_temp in enumerate(temp_list):
            for k,CP2_temp in enumerate(temp_list):
                for t,time in enumerate(time_list):
                    if f"CP1_{CP1_temp}_CP2_{CP2_temp}_{time:03}_{wind}" in filePath:
                            print(filePath)
                            rams_data = xr.open_dataset(outfilename)

                            u_prime = np.copy(rams_data["UPRIME_XAVG"])
                            v_prime = np.copy(rams_data["VPRIME_XAVG"])
                            w_prime = np.copy(rams_data["WPRIME_XAVG"])
                            tke = np.copy(rams_data["TKE_XAVG"])
                            rams_data.close()

                            uprime_dataset[w,c,k,t,:len(u_prime)] = u_prime
                            vprime_dataset[w,c,k,t,:len(v_prime)] = v_prime
                            wprime_dataset[w,c,k,t,:len(w_prime)] = w_prime
                            tke_dataset[w,c,k,t,:len(tke)] = tke
                            for time1 in range(len(tke_dataset[w,c,k,t,:,0,1])):
                                if time1 < cp2_start:
                                    cp_index = 0
                                else:
                                    cp_index = 1
                                cp_edge = n_indicies_array[w,c,k,t,time1,cp_index]
                                if np.isnan(cp_edge) == False:
                                    cp_edge = int(cp_edge)
                                    uprime_new = uprime_dataset[w,c,k,t,time1,:, cp_edge-50:cp_edge+51]
                                    uprime_cpedge[w,c,k,t,time1,:,:len(uprime_new[0]),cp_index] = np.copy(uprime_new)
                                    vprime_new = vprime_dataset[w,c,k,t,time1,:, cp_edge-50:cp_edge+51]
                                    vprime_cpedge[w,c,k,t,time1,:,:len(vprime_new[0]),cp_index] = np.copy(vprime_new)
                                    wprime_new = wprime_dataset[w,c,k,t,time1,:, cp_edge-50:cp_edge+51]
                                    wprime_cpedge[w,c,k,t,time1,:,:len(wprime_new[0]),cp_index] = np.copy(wprime_new)
                                    tke_new = tke_dataset[w,c,k,t,time1,:,cp_edge-50:cp_edge+51]
                                    tke_cpedge[w,c,k,t,time1,:,:len(tke_new[0]),cp_index] = np.copy(tke_new)
                            

## Plot Figure S4

In [ ]:
w = 0
c = 1
k =1 
t = -1
plt.figure(figsize = (30,20))
plt.subplot(421)
tke_plotted = tke_cpedge[w,c,k,t][:,:,:101,0]
uprime_plotted = uprime_cpedge[w,c,k,t][:,:,:101,0]
vprime_plotted = vprime_cpedge[w,c,k,t][:,:,:101,0]
wprime_plotted = wprime_cpedge[w,c,k,t][:,:,:101,0]
y = np.arange(1,2999)
vmin = 0
vmax = 13
c = plt.contourf(np.nanmean(tke_plotted[2:2+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,21,2))

plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))

xticks = np.arange(0,101,20)
xlabels = np.arange(-5,5.1,2)
plt.text(0.9,29, "a)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))
plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"CP1 -10K average TKE ($m^2$ $s^{-2}$)")
plt.ylabel("z (m)")
plt.subplot(423)
vmin = 0
vmax = 6
y = np.arange(1,2999)
c = plt.contourf(np.nanmean(uprime_plotted[2:2+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,21,2))

plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))
# plt.xlim(1100,2000)
plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"CP1 -10K average (u')$^{2}$ ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "c)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))
plt.ylabel("z (m)")
plt.subplot(425)
c = plt.contourf(np.nanmean(vprime_plotted[2:2+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,21,2))

plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))

plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"CP1 -10K average (v')$^{2}$ ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "e)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))
plt.ylabel("z (m)")
plt.subplot(427)
c = plt.contourf(np.nanmean(wprime_plotted[2:2+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,21,2))
plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))


plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"CP1 -10K average (w')$^{2}$ ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "g)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))
plt.ylabel("z (m)")

w = 0
c = 1
k =1 
t = -1
plt.subplot(422)
tke_plotted = tke_cpedge[w,c,k,t][:,:,:101,1]
uprime_plotted = uprime_cpedge[w,c,k,t][:,:,:101,1]
vprime_plotted = vprime_cpedge[w,c,k,t][:,:,:101,1]
wprime_plotted = wprime_cpedge[w,c,k,t][:,:,:101,1]
y = np.arange(1,2999)
c = plt.contourf(np.mean(tke_plotted[26:26+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,21,2))
plt.colorbar(c)
plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))
xticks = np.arange(0,101,20)
xlabels = np.arange(-5,5.1,2)
plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"$CP1_{-10K}CP2_{-10K}DT_{120MIN}$ average TKE ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "b)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))


plt.subplot(424)
y = np.arange(1,2999)
c = plt.contourf(np.mean(uprime_plotted[26:26+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,21,2))
plt.colorbar(c)
plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))
plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"$CP1_{-10K}CP2_{-10K}DT_{120MIN}$ average (u')$^{2}$ ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "d)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))

plt.subplot(426)
c = plt.contourf(np.mean(vprime_plotted[26:26+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,21,2))
plt.colorbar(c)
plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))
plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"$CP1_{-10K}CP2_{-10K}DT_{120MIN}$ average (v')$^{2}$ ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "f)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))

plt.subplot(428)
c = plt.contourf(np.mean(wprime_plotted[26:26+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,21,2))
plt.colorbar(c)
plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))
plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"$CP1_{-10K}CP2_{-10K}DT_{120MIN}$ average (w')$^{2}$ ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "h)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))

plt.show()
plt.close()

## Plot Figure S5

In [ ]:
w = 0
c = -1
k =-1 
t = -1
plt.figure(figsize = (30,20))
plt.subplot(421)
tke_plotted = tke_cpedge[w,c,k,t][:,:,:101,0]
uprime_plotted = uprime_cpedge[w,c,k,t][:,:,:101,0]
vprime_plotted = vprime_cpedge[w,c,k,t][:,:,:101,0]
wprime_plotted = wprime_cpedge[w,c,k,t][:,:,:101,0]
y = np.arange(1,2999)
vmin = 0
vmax = 13
c = plt.contourf(np.nanmean(tke_plotted[2:2+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,29,2))

plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))

xticks = np.arange(0,101,20)
xlabels = np.arange(-5,5.1,2)
plt.text(0.9,29, "a)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))
plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"CP1 -20K average TKE ($m^2$ $s^{-2}$)")
plt.ylabel("z (m)")
plt.subplot(423)
vmin = 0
vmax = 6
y = np.arange(1,2999)
c = plt.contourf(np.nanmean(uprime_plotted[2:2+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,29,2))

plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))

plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"CP1 -20K average (u')$^{2}$ ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "c)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))
plt.ylabel("z (m)")
plt.subplot(425)
c = plt.contourf(np.nanmean(vprime_plotted[2:2+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,29,2))

plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))

plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"CP1 -20K average (v')$^{2}$ ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "e)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))
plt.ylabel("z (m)")
plt.subplot(427)
c = plt.contourf(np.nanmean(wprime_plotted[2:2+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,29,2))

plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))


plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"CP1 -20K average (w')$^{2}$ ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "g)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))
plt.ylabel("z (m)")

w = 0
c = -1
k =-1 
t = -1

plt.subplot(422)
tke_plotted = tke_cpedge[w,c,k,t][:,:,:101,1]
uprime_plotted = uprime_cpedge[w,c,k,t][:,:,:101,1]
vprime_plotted = vprime_cpedge[w,c,k,t][:,:,:101,1]
wprime_plotted = wprime_cpedge[w,c,k,t][:,:,:101,1]
y = np.arange(1,2999)
c = plt.contourf(np.mean(tke_plotted[26:26+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,29,2))
plt.colorbar(c)
plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))
xticks = np.arange(0,101,20)
xlabels = np.arange(-5,5.1,2)
plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"$CP1_{-20K}CP2_{-20K}DT_{120MIN}$ average TKE ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "b)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))


plt.subplot(424)
y = np.arange(1,2999)
c = plt.contourf(np.mean(uprime_plotted[26:26+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,29,2))
plt.colorbar(c)
plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))
plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"$CP1_{-20K}CP2_{-20K}DT_{120MIN}$ average (u')$^{2}$ ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "d)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))
# plt.ylabel("z (m)")
plt.subplot(426)
c = plt.contourf(np.mean(vprime_plotted[26:26+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,29,2))
plt.colorbar(c)
plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))
plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"$CP1_{-20K}CP2_{-20K}DT_{120MIN}$ average (v')$^{2}$ ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "f)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))

plt.subplot(428)
c = plt.contourf(np.mean(wprime_plotted[26:26+10], axis =0)[1:,:], cmap = 'plasma', levels = np.arange(0,29,2))
plt.colorbar(c)
plt.ylim(0,30)
plt.yticks(np.arange(0,30,5), labels = np.arange(25,1525,250))
plt.xticks(xticks, labels = xlabels)
plt.xlabel("Distnace from CP edge (km)")
plt.title(r"$CP1_{-20K}CP2_{-20K}DT_{120MIN}$ average (w')$^{2}$ ($m^2$ $s^{-2}$)")
plt.text(0.9,29, "h)", weight = "bold", va = "top", fontsize = 45, bbox=dict(facecolor='white', edgecolor='none'))

plt.show()
plt.close()